##### 메일 보낼때 주석처리 풀고 보낼것. 

In [1]:
#!pip install selenium
#!pip install tabulate
#!pip install dart-fss
#!pip install opendartreader 
#from marcap import marcap_data
import datetime as datetime
from dateutil.parser import parse
today=datetime.date.today() #오늘날짜 정의
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import os 
from selenium import webdriver
from tabulate import tabulate as tabu
import requests 
import numpy as np
from io import BytesIO
from konlpy.tag import Okt
from konlpy.tag import Kkma
from konlpy.tag import Twitter
from konlpy.utils import pprint
import dart_fss as dartF
import io 
import zipfile
import xml.etree.ElementTree as et 
import json
import xmltodict
import urllib
import OpenDartReader as dart
#soup=BeautifulSoup(html, 'html.parser')

In [3]:
okt=Okt
kkma=Kkma
#사용 설명서 링크 https://dart-fss.readthedocs.io/en/latest/welcome.html
# https://opendart.fss.or.kr/ 
# Open DART API KEY 설정 api key 새로 받아와야 함 
today=datetime.date.today() #오늘날짜 정의
#df_marcap = marcap_data('2022-11-20', today.isoformat())
crtfc_key='8b9c4adb3537a255f5a2bda2a0e43f09eb29e495'
api_key='8b9c4adb3537a255f5a2bda2a0e43f09eb29e495'
dartF.set_api_key(api_key=api_key)
dart = dart(api_key) 
corp_list = dartF.get_corp_list() #다트에서 회사 리스트 받아옴 
np.set_printoptions(threshold=np.inf, linewidth=np.inf) #생략없이 dart fss프린트하게 하는 옵션
pd.set_option('display.max_rows', 500)  #데이터 프레임 안짤리고 전부 나오게 하는 명령어 
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 500)

Output()

Output()

Output()

##### 시가총액 기능 x. 사용하기 위해선 marcap 데이터 (5G) 다운로드 필요

In [28]:
import nltk

print('원하는 회사 이름을 입력하세요 (영어는 대문자로 입력해주세요.)')

user_message=input()

corp_name=corp_list.find_by_corp_name(user_message, exactly=True)[0]

print(corp_name)

print("해당숫자는 회사코드입니다. 입력해주세요")

corp_code_input=input()

print("해당 회사에서 알고 싶은 정보를 입력해주세요 (재무, 배당, 거래소공시내역, 자기주식처분,시가총액 등)")
      
user_message2=input()


#Q1=okt().nouns(user_message2) #형태소분석기로 명사만 뽑기 사용한것  데이터 형태는 리스트임 
#유형자산을 하나로 인식하는 형태소분석기 필요  kkma만 유형자산 입력이 유형과 자산을 따로 보지 않고 '유형자산'으로 인식함 
Q1=kkma().nouns(user_message2)

#https://infinitt.tistory.com/21 조건문 관련 링크 
#elif i== '재무' or '재무제표' <<이렇게 하면  i가 문자열이나 문자열이면 조건을 만족한다라는 의미가 되어버림. (따라서 무조건 조건을 만족하게 되어 그 조건만 출력됨)
#정확히 내가 원하는 문자를 가지고 있는지 묻기 위해선 따로 i==지정을 해줘야만 함. 

for i in Q1 :
    if  i == '배당' or i == '배당금':
        print(dart.report(user_message, '배당', 2021, reprt_code='11011'))
        
    elif i == '재무' or i == '재무제표' :
        data=dart.finstate_all(corp=user_message, bsns_year=2021, reprt_code='11011', fs_div='CFS') #pandas 데이터 프레임임 
        data2=data.loc[:,['bsns_year','sj_nm','account_nm','thstrm_nm','thstrm_amount','currency']] #특정 열만 추출함 (현재 이번년도만 추출)
       # print(data2)
        print(tabu(data2, headers='keys', tablefmt='psql', showindex=True))
    
    elif i == '자기주식처분' or i == '처분' or i =='사주' or i =='자사주':
        report=dart.event(user_message, '자기주식처분', start=2021, end=today)
        report_ad=report.loc[:,['corp_name','dppln_stk_ostk','dpstk_prc_ostk','dppln_prc_ostk','dpprpd_edd','dp_pp']] #특정 열 추출 
        report_ad.colums= ['corp_name','dppln_stk_ostk','dpstk_prc_ostk','dppln_prc_ostk','dpprpd_edd','dp_pp'] #특정 열 추출 
        report_ad.rename(columns={'dppln_stk_ostk':'주식 수','dpstk_prc_ostk':'주당가격','dppln_prc_ostk':'처분총액','dp_pp':'처분목적'}, inplace=True) #열의 이름 바꿔줌 
       # print(report_ad)
        print(tabu(report_ad, headers='keys', tablefmt='psql', showindex=True))
                   
    elif i == '공시' or i == '공시정보' or i=='내역' or i=='공시내역' or i=='목록':
        report_list=dartF.filings.search(corp_code=corp_code_input, bgn_de=20200101, end_de=None, last_reprt_at='N', pblntf_ty='I',
                                         pblntf_detail_ty='H', corp_cls=None, sort='date', sort_mth='asc', page_no=1, page_count=100)
        
        print(report_list)  #이새끼 이쁘게 출력 어렵나?
        #print(tabu(report_list, headers='keys', tablefmt='psql', showindex=True))
  """      
    elif i == '시가총액' or i == '시총':
        marketcap = df_marcap[df_marcap['Name']==user_message]
        marketcap_ad= marketcap.loc[:,['Name','Marcap','Market']]
        #print(marketcap_ad)
        print(tabu(marketcap_ad, headers='keys', tablefmt='psql', showindex=True))
    """    
    else:
        print("질문을 이해하지 못했습니다")


원하는 회사 이름을 입력하세요 (영어는 대문자로 입력해주세요.)


 SK하이닉스


[00164779]SK하이닉스
해당숫자는 회사코드입니다. 입력해주세요


 00164779


해당 회사에서 알고 싶은 정보를 입력해주세요 (재무, 배당, 거래소공시내역, 자기주식처분,시가총액 등)


 시가총액 얼마?


질문을 이해하지 못했습니다
+---------------------+------------+----------------+----------+
| Date                | Name       |         Marcap | Market   |
|---------------------+------------+----------------+----------|
| 2022-11-21 00:00:00 | SK하이닉스 | 62826604099500 | KOSPI    |
| 2022-11-22 00:00:00 | SK하이닉스 | 62025801498000 | KOSPI    |
| 2022-11-23 00:00:00 | SK하이닉스 | 62025801498000 | KOSPI    |
+---------------------+------------+----------------+----------+
질문을 이해하지 못했습니다
질문을 이해하지 못했습니다
